In [3]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from dask.distributed import Client, progress, wait
import attengineshell as ate
import numpy as np
import dask.array as da
import dask
# from dask import delayed

import sys

sys.path.insert(0, '../thirdparty/singleTrace_d2geo/')
import CompleTrace as cmpAttr

# cd ../thirdparty/Semblance
# python setup.py install
from semblance import semblance

# from semblance import semblance

/glb/data/cdis_projects/users/usnrej/attEngine/python-env/lib/python3.5/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [50]:
wx = 1
wy = 1
wz = 1
def nsemblance(f):
#     print(f.shape)
    nx=f.shape[0]
    ny = f.shape[1]
    nz = f.shape[2]

#     output  = np.zeros([nx, ny, nz])
# #     output[0] = f.shape[0]
#     output[1] = f.shape[1]
# #     output[2] = f.shape[2]
#     return output
    
    output  = np.zeros([nx-2*wx, ny-2*wy, nz-2*wz])
#     output  = np.zeros([nx, ny, nz])
    for iz in range(wz,nz-wz):
        for iy in range(wy,ny-wy):
            for ix in range(wx,nx-wx):

                xfrom = max(ix - wx, 0)
                xto   = min(ix + wx + 1, nx)

                yfrom = max(iy - wy, 0)
                yto   = min(iy + wy + 1, ny)

                zfrom = max(iz - wz, 0)
                zto   = min(iz + wz + 1, nz)
                
#                 print("---")
#                 print(ix,iy,iz)
#                 print(xfrom,xto)
#                 print(yfrom,yto)
#                 print(zfrom,zto)

                sumSqrSum = 0.0
                sumSumSqr = 0.0

                for k in range(zfrom,zto):
                    sum1 = 0
                    sum2 = 0
                    nuvalid = 0
                    for i in range(xfrom,xto):
                        for j in range(yfrom,yto):
                            value = f[i,j,k]
#                             if (value == value): 
                            nuvalid +=1
                            sum1 += value
                            sum2 += value*value

                    sumSqrSum+=sum1*sum1
                    sumSumSqr+=sum2*nuvalid

                output[ix-wx,iy-wy,iz-wz] = 0 if sumSumSqr == 0 else 1 - sumSqrSum / sumSumSqr

    return output

data = np.arange(4*4*6).reshape((4, 4, 6))
data = data.astype(np.float32)
print(data.shape, range(wz,4-wz+1), list(range(wx,4-wx)), list(range(wz,5-wz)))
c=nsemblance(data)
c[:,:,0]
# print(c)
# # print(nsemblance(a))
# b=semblance(a)
# print(b)

(4, 4, 6) range(1, 4) [1, 2] [1, 2, 3]


array([[0.2978827 , 0.22951434],
       [0.11882342, 0.09879732]])

In [15]:
# Original
chunkdata = da.from_array(data, chunks=data.shape)
decomposed_data = da.overlap.overlap_internal(chunkdata, {0:wx,1:wy,2:wz})
daskout = decomposed_data.map_blocks(semblance)
result = da.overlap.trim_internal(da.overlap.add_dummy_padding(daskout,{0:wx,1:wy,2:wz},
                                                           {0:'none',1:'none',2:'none'}),{0:wx,1:wy,2:wz})
print(result.compute())

[[[0.33359498 0.33359498 0.33359498 0.33359498 0.33359498]
  [0.2861815  0.2861815  0.2861815  0.2861815  0.2861815 ]
  [0.19358407 0.19358407 0.19358407 0.19358407 0.19358407]
  [0.14996472 0.14996472 0.14996472 0.14996472 0.14996472]]

 [[0.31184536 0.31184536 0.31184536 0.31184536 0.31184536]
  [0.2793296  0.2793296  0.2793296  0.2793296  0.2793296 ]
  [0.21639511 0.21639511 0.21639511 0.21639511 0.21639511]
  [0.18627004 0.18627004 0.18627004 0.18627004 0.18627004]]

 [[0.1210184  0.1210184  0.1210184  0.1210184  0.1210184 ]
  [0.11359081 0.11359081 0.11359081 0.11359081 0.11359081]
  [0.09478144 0.09478144 0.09478144 0.09478144 0.09478144]
  [0.08409943 0.08409943 0.08409943 0.08409943 0.08409943]]

 [[0.03451356 0.03451356 0.03451356 0.03451356 0.03451356]
  [0.03464318 0.03464318 0.03464318 0.03464318 0.03464318]
  [0.02944145 0.02944145 0.02944145 0.02944145 0.02944145]
  [0.02489165 0.02489165 0.02489165 0.02489165 0.02489165]]]


In [57]:
# # Mine
# chunkdata = da.from_array(data, chunks=(2,2,2))
depth = {0:wx,1:wy,2:wz}
# boundary = {0:'none', 1:'none',2:'none'}
boundary = {0:0, 1:0,2:0}
# # boundary = {0:0, 1:0}
# daskout = da.map_overlap(chunkdata, nsemblance, depth=depth, boundary=boundary, trim=True, dtype=np.float32)
# # result = da.overlap.trim_internal(da.overlap.add_dummy_padding(daskout,{0:wx,1:wy,2:wz},
# #                                                            {0:'none',1:'none',2:'none'}),{0:wx,1:wy,2:wz})
# print(daskout.compute())

# Mine
chunkdata = da.from_array(data, chunks=(2,2,2))
daskout = da.map_overlap(chunkdata, nsemblance, depth=depth, boundary=boundary, trim=False, dtype=np.float32)
b = daskout.compute()
print(daskout.compute())
print("===")

daskout = da.map_overlap(chunkdata, semblance, depth=depth, boundary=boundary, trim=True, dtype=np.float32)
print(daskout.compute()-b)

[[[0.72836369 0.72154416 0.70917001 0.69791424 0.68767991 0.68301572]
  [0.55272609 0.54478708 0.53028527 0.51704028 0.50493701 0.49938226]
  [0.47907178 0.47445051 0.46593002 0.45810125 0.45089806 0.44756077]
  [0.63032191 0.628076   0.62392017 0.62009209 0.61656034 0.61491828]]

 [[0.56125852 0.55580715 0.54553636 0.53585846 0.52673885 0.52243474]
  [0.30481883 0.2978827  0.28478362 0.27242377 0.26075841 0.25523929]
  [0.2344154  0.22951434 0.2202231  0.21143548 0.20311981 0.19917012]
  [0.46745158 0.46474948 0.45961868 0.4547608  0.45015854 0.44796908]]

 [[0.4193619  0.41791312 0.41514016 0.41249538 0.4099714  0.40876158]
  [0.12076365 0.11882342 0.1151025  0.11154652 0.10814632 0.10651351]
  [0.10028266 0.09879732 0.0959398  0.09320034 0.09057274 0.08930721]
  [0.39253568 0.39168956 0.39005966 0.3884951  0.38699254 0.38626797]]

 [[0.57180212 0.57155681 0.5710854  0.57063428 0.57020231 0.56999446]
  [0.35773155 0.35737971 0.35670237 0.35605292 0.35542987 0.35512958]
  [0.3539804  

In [86]:
x = np.arange(16).reshape((4, 4))
chunkdata = da.from_array(x, chunks=(2, 2))
print(x)
print(chunkdata.chunks)
print(chunkdata.blocks[0].compute())

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
((2, 2), (2, 2))
[[0 1 2 3]
 [4 5 6 7]]


In [83]:
def func(block):
#     print("block")
    s = np.sum(block)
    return block + s
#     return gaussian_filter(block, sigma=1)
wx, wy = 0, 0
depth = {0:wx,1:wy}
# boundary = {0:'none', 1:'none'}
boundary = {0:0, 1:0}
decomposed_data = da.overlap.overlap(chunkdata, depth=depth, boundary=boundary)
daskout = decomposed_data.map_blocks(func)
# result = da.overlap.trim_internal(da.overlap.add_dummy_padding(daskout,
#                                     {0:wx,1:wy},{0:'none',1:'none'}),
#                                     {0:wx,1:wy})
# result = da.overlap.trim_internal(daskout, depth)
result = da.overlap.trim_internal(da.overlap.add_dummy_padding(daskout,
                                    depth, boundary),
                                    depth)
print(chunkdata.chunks)
# print(chunkdata.blocks[0].compute(), np.sum(chunkdata.blocks[0].compute()))
# print(chunkdata.blocks[1].compute(), np.sum(chunkdata.blocks[1].compute()))
print(result.blocks[0].compute())
# print(chunkdata.blocks[3].compute())

a = da.map_overlap(chunkdata, func, depth=depth, boundary=boundary, trim=True)
print(a.blocks[0].compute())

((2, 2), (2, 2))
[[10 11 20 21]
 [14 15 24 25]]
[[10 11 20 21]
 [14 15 24 25]]


In [135]:
def func(block):
#     print("block")
    s = np.sum(block)
    return block + s
#     return gaussian_filter(block, sigma=1)
wx, wy = 1, 1
depth = {0:wx,1:wy}
# boundary = {0:'none', 1:'none'}
boundary = {0:'none', 1:'none'}
daskoutpad = da.overlap.add_dummy_padding(chunkdata, depth, boundary)
daskoutmap = daskoutpad.map_blocks(func)
daskout = da.overlap.trim_internal(daskoutmap, depth)
# print(daskout.blocks[0].compute())
print(np.array(daskoutpad), daskoutpad.shape)

[[             0 47500194118032 47500194056016 47500194118104
  47500194179256              0]
 [             3              0              1              2
               3              3]
 [             6              4              5              6
               7              6]
 [             0              8              9             10
              11              0]
 [             3             12             13             14
              15              3]
 [             6 47500194118032 47500194056016 47500194118104
  47500194179256              6]] (6, 6)


In [130]:
dask.__version__

'1.2.1'

In [71]:
print(x)
print(np.array(result)-x)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[[45 45 54 54]
 [45 45 54 54]
 [81 81 90 90]
 [81 81 90 90]]


In [72]:
10+2+6+10+9+8

45